In [2]:
import pandas as pd
import seaborn as sns
import nltk
import pymorphy2
import numpy as np
import re
from tqdm import tqdm
import matplotlib.pyplot as plt
from catboost import CatBoostClassifier
from pandarallel import pandarallel
from scipy import sparse
from nltk.corpus import stopwords
import tensorflow as tf
from datasets import Dataset

pandarallel.initialize(progress_bar=False)
tqdm.pandas()
sns.set(rc={'figure.figsize':(18,12)})
morph = pymorphy2.MorphAnalyzer()

pd.set_option('display.max_columns', 100)

INFO: Pandarallel will run on 1 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [3]:
df = pd.read_excel('Data_scientist_NLP_.xlsx')
df = df.dropna()
df

,Дата,Категория,Тема,Муниципалитет,Текст
0,2022-04-02 15:43:24.726,Вопрос,Благоустройство,Хайбуллинский р-н,'Вид из окна потрясающий (пр. С. Юлаева напрот...
1,2022-05-06 14:46:31.109,Вопрос,Социальное обслуживание и защита,Брянск г,"'[id294938415:bp-88767010_4|Αнастасия], Здравс..."
2,2022-01-14 07:20:51.308,Вопрос,Социальное обслуживание и защита,Улан-Удэ г,'[club53367440:bp-53367440_121743|Министерство...
3,2022-05-18 03:54:37.952,Вопрос,Социальное обслуживание и защита,Улан-Удэ г,'Здравствуйте когда будет выплата ДВ?
4,2022-05-20 07:33:30.558,Вопрос,Социальное обслуживание и защита,Челябинск г,"'Девочки всем привет,у меня такой вопрос получ..."
...,...,...,...,...,...
29544,2022-04-11 08:42:35.514,Вопрос,Общественный транспорт,Октябрьский г,'Добрый день.Почему подорожала плата за проезд...
29545,2022-05-04 07:38:28.007,Вопрос,"МФЦ ""Мои документы""",Миасс г,'Миасс. Девочки подскажите номер телефона пф о...
29546,2022-04-15 19:15:52.086,Вопрос,Социальное обслуживание и защита,Уфа г,'Здравствуйте почему пришел отказ?<br>db7ced5b...
29547,2022-07-11 07:51:28.096,Вопрос,Общественный транспорт,Белгород г,'А НАЗАД С КАКИМ ИНТЕРВОЛОМ БУДУТ ЕЗДИТЬ АВТОБ...


In [4]:
def text_preproses(row):
    text = row['Текст']
    text = text.lower()
    text = text.replace("'", '')
    text = nltk.word_tokenize(text, language="russian")
    text = list(map(lambda w: morph.parse(w)[0].normal_form, text))
    tags = []
    [tags.extend(str(morph.parse(w)[0].tag).split(',')) for w in text]
    text = ' '.join(text)
    row['tags'] = ' '.join(tags)
    row['Clean_text'] = text
    return row

df = df.parallel_apply(text_preproses, axis=1)

In [5]:
from transformers import BertTokenizer, TFBertModel
from datasets import Dataset

In [6]:
tokenizer = BertTokenizer.from_pretrained('sberbank-ai/sbert_large_mt_nlu_ru')
model = TFBertModel.from_pretrained("sberbank-ai/sbert_large_mt_nlu_ru")

All model checkpoint layers were used when initializing TFBertModel.

All the layers of TFBertModel were initialized from the model checkpoint at sberbank-ai/sbert_large_mt_nlu_ru.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [7]:
def tokinize_func(examples):
    model_inputs = tokenizer(examples['Clean_text'])
    return model_inputs

dataset = Dataset.from_pandas(df[['Clean_text']])
dataset = dataset.map(tokinize_func, batched=True)
tf_dataset = model.prepare_tf_dataset(dataset, batch_size=16, shuffle=False, tokenizer=tokenizer)

  0%|          | 0/30 [00:00<?, ?ba/s]

In [8]:
for i, b in enumerate(tqdm(tf_dataset)):
    output = model(b)[1]
    if i == 0:
        outputs = output
    else:
        outputs = tf.concat([outputs, output], 0)

100%|██████████| 1847/1847 [19:11<00:00,  1.60it/s]


In [9]:
np.save('text_vectors.npy', outputs.numpy())
np.load('text_vectors.npy').shape

(29538, 1024)

In [10]:
outputs.shape

TensorShape([29538, 1024])